<a href="https://colab.research.google.com/github/Kewin90/1data_test/blob/master/Lesson_3_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
# !pip install pyspark
# !pip install install-jdk
# !pip install findspark
# Вышеуказанный код уже выполнен

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import random
import pandas as pd

spark = SparkSession.builder \
    .appName("Student Data") \
    .getOrCreate()

# Создание данных о студентах
num_students = 100
students = []
for i in range(num_students):
    student_id = i + 1
    name = f"Student_{student_id}"
    age = random.randint(18, 25)
    city = random.choice(["Moscow", "Saint Petersburg", "Novosibirsk", "Yekaterinburg", "Nizhny Novgorod"])
    students.append((student_id, name, age, city))

# Создание данных о предметах
num_courses = 5
courses = []
course_names = ["Mathematics", "Physics", "Chemistry", "Biology", "History"]
for i in range(num_courses):
    course_id = i + 1
    name = course_names[i]
    description = f"This is a course about {name}."
    courses.append((course_id, name, description))

# Создание данных об оценках
grades = []
for student_id in range(1, num_students + 1):
    for course_id in range(1, num_courses + 1):
        grade = random.randint(1, 10)
        grades.append((student_id, course_id, grade))

# Преобразование в DataFrame
students_df = spark.createDataFrame(students, ["student_id", "name", "age", "city"])
courses_df = spark.createDataFrame(courses, ["course_id", "name", "description"])
grades_df = spark.createDataFrame(grades, ["student_id", "course_id", "grade"])
# Показать созданные таблицы DataFrame
students_df.show(10)
courses_df.show(10)
grades_df.show(10)
# Подсчет среднего возрата студентов
average_age = students_df.agg(F.avg("age")).first()[0]
print(f"Средний возраст студентов: {average_age}")

# Количество студентов из каждого города
students_per_city = students_df.groupBy("city").agg(F.count("student_id").alias("student_count"))
students_per_city.show()

# Подсчет среднего балла студентов по всем предметам
average_grade = grades_df.agg(F.avg("grade")).first()[0]
print(f"Средний балл студентов по всем предметам: {average_grade}")

# Самый популярный предмет, но он постоянен потому что все студенты выбрали все курсы
popular_course = grades_df.groupBy("course_id").agg(F.count("student_id").alias("student_count")) \
    .orderBy(F.desc("student_count")).first()
course_name = courses_df.filter(courses_df.course_id == popular_course[0]).select("name").first()[0]
print(f"Самый популярный предмет: {course_name} (Количество студентов: {popular_course[1]})")


spark.stop()



+----------+----------+---+----------------+
|student_id|      name|age|            city|
+----------+----------+---+----------------+
|         1| Student_1| 20|   Yekaterinburg|
|         2| Student_2| 25| Nizhny Novgorod|
|         3| Student_3| 20|     Novosibirsk|
|         4| Student_4| 19|   Yekaterinburg|
|         5| Student_5| 24|     Novosibirsk|
|         6| Student_6| 20|     Novosibirsk|
|         7| Student_7| 24|Saint Petersburg|
|         8| Student_8| 25|   Yekaterinburg|
|         9| Student_9| 25|          Moscow|
|        10|Student_10| 24|          Moscow|
+----------+----------+---+----------------+
only showing top 10 rows

+---------+-----------+--------------------+
|course_id|       name|         description|
+---------+-----------+--------------------+
|        1|Mathematics|This is a course ...|
|        2|    Physics|This is a course ...|
|        3|  Chemistry|This is a course ...|
|        4|    Biology|This is a course ...|
|        5|    History|This i